# DewanLab DeepLabCut Video Analysis 

## 1. Import Dependencies
#### *You can ignore any errors from tensorflow about oneDNN, cuBLAS, libnvifer, or TensorRT*

In [ ]:
import os
from pathlib import Path
os.environ['DLClight']="True"
import deeplabcut
from pandas import pd
print("Dependencies successfully imported!")


# 2. Set User Configurables

In [ ]:
## User Configuratbles
"""
- video_dir: Path to EPM videos
- video_file_extensions: list of potential video file extensions without the '.'
- recursive_video_search: set to *True* to recursively search all folders in {video_dir} for video files
- output_threshold: p-value above which to consider a "good" frame when checking quality of a labeled video
- _shuffle: which model shuffle to use
- _save_as_csv: whether or not to output a csv for each processed video
- _trailpoints: number of colored points to trail behind the marked points
"""

video_dir = '/blue/adamdewan/EPM/VGLUT-20/'
video_file_extensions = ['avi', 'mp4', 'mkv']
recursive_video_search = False

output_threshold = 0.8

_shuffle = 1
_save_as_csv = True
_trailpoints = 0


## 3. Get Videos

In [ ]:
# The notebook should be in the root directory of the project
current_dir = Path.cwd()
config_path = current_dir.joinpath('config.yaml')
new_videos_dir = Path(video_dir)

if not new_videos_dir.exists():
    raise FileNotFoundError(f'The path {{{new_videos_dir}}} does not exist!')

# Get a list of the videos in "New Video Files"
video_paths = []

for file_extension in video_file_extensions:
    search_string = f'*.{file_extension}'
    if recursive_video_search:
        new_vids = list(new_videos_dir.rglob(search_string))
    else:
        new_vids = list(new_videos_dir.glob(search_string))

    if len(new_vids) > 0:
        video_paths.extend(new_vids)
        
video_paths = [video for video in video_paths if 'labeled' not in video.name]
# Filter out any analyzed videos

video_strings = [str(video) for video in video_paths]

print(f'Found the following video(s): {video_strings}')


# 4. Process Videos

In [ ]:
for video in video_paths:
    try:
        video_type = video.suffix
        deeplabcut.analyze_videos(str(config_path), str(video), shuffle=_shuffle, save_as_csv=_save_as_csv, videotype=video_type)
        deeplabcut.create_labeled_video(str(config_path), str(video), shuffle=_shuffle, videotype=video_type, trailpoints=_trailpoints)
    except Exception as e:
        print(f"An error has occurred while processing video {{{video}}}")
        print(e)
        

# 5. (Optional) Analyze Analysis Output

In [ ]:
# Get H5 Files
print(f"Percentile of frames at or above threshold ({output_threshold}) for each video:")

for video in video_paths:
    video_stem = video.stem
    h5_file = list(video.parent.glob(f'{video_stem}*.h5'))[0]

    df = pd.read_hdf(h5_file)
    # Get likelihood for each component, sum the values above the threshold, and calculate percentile
    head_like_mask = (df[df.columns[2]] >= output_threshold)
    head_like_percentile = (head_like_mask.sum() * 100) / len(head_like_mask)
    body_like_mask = (df[df.columns[5]] >= output_threshold)
    body_like_percentile = (body_like_mask.sum() * 100) / len(body_like_mask)
    
    result = f'Video: {video_stem}\nHead Percentile: {round(head_like_percentile, 2)}%, Body Percentile: {round(body_like_percentile, 2)}%\n'
    print(result)
